Keras Callbacks :

Keras Callbacks are powerful tools used during model training to control their execution and improve their performance. They can be thought of as functions that are automatically executed at different stages of training, such as the beginning or end of each epoch, or after updating a batch of data.

Why do we use Callbacks?

EarlyStopping: Stops training if performance has not improved after a certain number of epochs.

ModelCheckpoint: Saves the best version of the model during training.

ReduceLROnPlateau: Reduces the learning rate when optimization stops.

TensorBoard: Record and visually monitor training data.










---

Using Callbacks during Training

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Download California Housing data
dataset = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(dataset.data, dataset.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [3]:
# Data normalization (improve performance by bringing values ​​within an appropriate range)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [4]:
# Create a Wide & Deep Network model
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [5]:
# Model compilation model.compile()
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [6]:
# Prepare the data to fit the input
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [7]:
# Train the model with Early Stopping and save the best model
# The goal of EarlyStopping is to stop model training when performance on the validation set does not improve for a certain period
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_checkpoint.keras", save_best_only=True)
# ModelCheckpoint : save the best model achieved during training. Here, save_best_only=True is specified which means that the model will be saved only if performance improves on the validation set
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
# patience=3 : If the model does not improve its performance over 3 epochs, we stop training to avoid overtraining or overlearning
# restore_best_weights=True means that when training stops, the best weights of the model (which achieved the best accuracy or performance during the training period) will be retrieved
"""
The goal of the callbacks:
The first callback (checkpoint_cb) is used to save the best model achieved during training.
The second callback (early_stopping_cb) is used to stop training early if the performance doesn't improve over a certain number of epochs (in this case, 3 epochs).
"""
history = model.fit(
    (X_train_A, X_train_B), y_train, epochs=20,
    validation_data=((X_valid_A, X_valid_B), y_valid),
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 3.2818 - val_loss: 4.3430
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.9533 - val_loss: 1.1293
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.7291 - val_loss: 0.6764
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.6678 - val_loss: 0.5903
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.6206 - val_loss: 0.5554
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.5627 - val_loss: 0.5254
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5435 - val_loss: 0.4993
Epoch 8/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.5340 - val_loss: 0.4798
Epoch 9/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4992 - val_loss: 0.4651
Epoch 10/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4731 - val_loss: 0.4493
Epoch 11/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.4728 - val_loss: 0.4362
Epoch 12/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/ste

In [8]:
# Retrieve the best saved model
tuned_model = keras.models.load_model("my_checkpoint.keras")

In [9]:
# Evaluate the model on test data
mse_test = tuned_model.evaluate((X_test_A, X_test_B), y_test)

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4313


In [10]:
# Predicting some new values
y_pred = tuned_model.predict((X_new_A, X_new_B))
print("Predictions:", y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Predictions: [[0.3465656]
 [1.8708789]
 [3.200811 ]]
